In [4]:
import numpy as np
import pandas as pd
df = pd.read_csv('../Cases/Concrete Strength/Concrete_Data.csv')
df.head()

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
X, y = df.drop('Strength', axis=1), df['Strength']

In [20]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

from sklearn.tree import DecisionTreeRegressor
dct = DecisionTreeRegressor()

from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True, random_state=24)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(lr, X, y, cv=kfold, scoring='r2')
np.mean(scores)

0.6066065590942402

In [21]:
from sklearn.linear_model import Ridge
ridge = Ridge()
results = cross_val_score(ridge, X, y, cv=kfold, scoring='r2')
results.mean()

0.6066065614459983

In [36]:
alpha = [0.01, 0.1, 0.5, 1, 1.5, 2]
scores = []
for i in alpha:
    ridge = Ridge(alpha=i)
    results = cross_val_score(ridge, X, y, cv=kfold, scoring='r2')
    scores.append(np.mean(results))

print(np.max(scores))
print(alpha[np.argmax(scores)])

0.6066065637413789
2


In [40]:
import warnings
warnings.filterwarnings('ignore')

## Grid Search

### Ridge

In [41]:
from sklearn.model_selection import GridSearchCV
params = {'alpha': [0.01, 0.1, 0.5, 1, 1.5, 2]}
ridge = Ridge()
gcv = GridSearchCV(ridge,  param_grid=params, cv=kfold, scoring='r2')
gcv.fit(X, y)
print(gcv.best_params_, gcv.best_score_)

{'alpha': 2} 0.6066065637413789


### ElasticNet

In [66]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=24)
params = {'alpha': [0.01, 0.1, 0.5, 1, 1.5, 2], 'l1_ratio': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]}
en = ElasticNet()
gcv = GridSearchCV(en,  param_grid=params, cv=kfold, scoring='r2')
gcv.fit(X, y)
print(gcv.best_params_, gcv.best_score_)

{'alpha': 0.01, 'l1_ratio': 0.0001} 0.6066065767173022


## Inferencing

In [60]:
test = pd.read_csv('../Cases/Concrete Strength/testConcrete.csv')
test.head(1)

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age
0,495,120,0,155,5,866,884,75


In [61]:
X.head(1)

,Cement,Blast,Fly,Water,Superplasticizer,Coarse,Fine,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28


In [62]:
en = ElasticNet(alpha=0.01, l1_ratio=0.0001)     # en = ElasticNet(alpha=0.01, l1_ratio=0.0001)
en.fit(X, y)
y_pred = en.predict(test)
# test['Strength'] = y_pred
# test

In [63]:
gcv.predict(test)

array([68.73885875, 31.78890048, 19.49349433, 46.84958715, 58.04431979,
       16.09657772, 50.35218236, 80.31328544, 31.13719466, 41.57101744,
       43.44500823, 60.77350571, 52.37080651, 15.38705685])

In [64]:
best_model = gcv.best_estimator_
best_model.predict(test)

array([68.73885875, 31.78890048, 19.49349433, 46.84958715, 58.04431979,
       16.09657772, 50.35218236, 80.31328544, 31.13719466, 41.57101744,
       43.44500823, 60.77350571, 52.37080651, 15.38705685])